In [53]:
%load_ext autoreload
%autoreload 2

from src.resnet import ResNet, resnet152
from src.mobilenet_v2 import InvertedResidual, MobileNetV2, mobile_net_config
import torch
import torch.nn as nn
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [54]:
def calculate_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

### Input example

In [55]:
x = torch.randn(2, 3, 1024, 1024)

## Resnet

In [56]:
num_blocks = [3, 4, 6, 3] # resnet50
resnet = ResNet(num_blocks=num_blocks, in_channels=3, num_classes=1000)
out = resnet(x)
assert out.shape == (2, 1000)
print(f"Number of parameters in resnet: {calculate_params(resnet)}")

Number of parameters in resnet: 25575912


## Mobilenet v2

In [57]:
# inverted residual
w = 112
h = 112
for_inverted = torch.randn(2, 16, 112, 112)
in_ch = 16
out_ch = 24
inverted_residual = InvertedResidual(in_ch, out_ch, expansion=6, stride=2)
assert inverted_residual(for_inverted).shape == (2, out_ch, w // 2, h // 2)

In [58]:
mobile_net_config # t, c, n, s

[[1, 16, 1, 1],
 [6, 24, 2, 2],
 [6, 32, 3, 2],
 [6, 64, 4, 2],
 [6, 96, 3, 1],
 [6, 160, 3, 2],
 [6, 320, 1, 1]]

In [59]:
# mobilenet_v2
mobile_net = MobileNetV2(mobile_net_config, in_channels=3, num_classes=1000)
out = mobile_net(x)
assert out.shape == (2, 1000)
print(f"Number of parameters in mobilenet: {calculate_params(mobile_net)}")

Number of parameters in mobilenet: 3521928


In [52]:
# mobilenet_v2
mobile_net = MobileNetV2(mobile_net_config, in_channels=3, num_classes=1000)
out = mobile_net(x)
assert out.shape == (2, 1000)
print(f"Number of parameters in mobilenet: {calculate_params(mobile_net)}")

Number of parameters in mobilenet: 3523048


In [60]:
mobile_net

MobileNetV2(
  (init_conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (init_bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (init_activation): ReLU6(inplace=True)
  (init_block): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU6(inplace=True)
  )
  (blocks): Sequential(
    (0): InvertedResidual(
      (init_layer): Identity()
      (depthwise_conv): Sequential(
        (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
      )
      (linear): Sequential(
        (0): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1))
        (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): 